# Convolutional neural networks (CNN) for CIFAR-10/100 using MLX

Markus Enzweiler, markus.enzweiler@hs-esslingen.de

This is a demo used in a Computer Vision & Machine Learning lecture. Feel free to use and contribute.

We build and train a CNN for CIFAR-10 / CIFAR-100 image classification, see https://www.cs.toronto.edu/~kriz/cifar.html. We use the Python code from https://github.com/menzHSE/mlx-cifar-10-cnn.git and execute it via this notebook. 


**Note: This requires a machine with an Apple SoC, e.g. M1/M2/M3 etc.**

See: https://github.com/ml-explore/mlx

## Setup

Adapt `packagePath` to point to the directory containing this notebeook.

In [ ]:
# Imports
import sys
import os
import threading
import subprocess
import fcntl
import errno

In [ ]:
# Package Path
package_path = "./" # local
print(f"Package path: {package_path}")


def check_for_colab():
  try:
      import google.colab
      return True
  except ImportError:
      return False

# Running on Colab?
on_colab = check_for_colab()

In [ ]:
# Clone git repository

# Absolute path of the repository directory
repo_dir = os.path.join(package_path, "mlx-cifar-10-cnn")
repo_url = "https://github.com/menzHSE/mlx-cifar-10-cnn.git"

# Store the original working directory
original_cwd = os.getcwd()

# Check if the directory already exists using the absolute path
if os.path.exists(os.path.join(original_cwd, repo_dir)):
    print("Repository exists. Resetting to HEAD...")
    # Navigate into the repository directory
    os.chdir(repo_dir)
    # Fetch the latest changes from the remote
    subprocess.run(["git", "fetch", "origin"])
    # Reset the local branch to the latest commit from the remote
    subprocess.run(["git", "reset", "--hard", "origin/HEAD"])
    # Change back to the original working directory
    os.chdir(original_cwd)
else:
    print("Cloning repository...")
    # Clone the repository if it doesn't exist
    subprocess.run(["git", "clone", repo_url, repo_dir])


In [ ]:
# Install requirements in the current Jupyter kernel
req_file = os.path.join(repo_dir, "requirements.txt")
if os.path.exists(req_file):
    !{sys.executable} -m pip install -r {req_file}
else:
    print(f"Requirements file not found: {req_file}")

## Functions to interface with the code in the repository

In [ ]:
def execute(script_name, params=None):
    if on_colab:
        executeCaptureColab(script_name, params)
    else:
        executeCapture(script_name, params)

def executeCapture(script_name, params=None):
    script_path = os.path.join(repo_dir, script_name)
    if os.path.exists(script_path):
        print(f"Executing script: {script_path}")
        # Create the command list starting with Python and the script path
        command = ["python", script_path]
        # Add additional arguments from the params dictionary
        if params:
            for key, value in params.items():
                command.append(f"--{key}")
                command.append(str(value))
        print(command)
        subprocess.run(command)
    else:
        print(f"Script not found: {script_path}")

# This is very hacky ... but it's hard to capture the output of a subprocess in Colab
def executeCaptureColab(script_name, params=None):
    script_path = os.path.join(repo_dir, script_name)
    if os.path.exists(script_path):
        print(f"Executing script: {script_path}")
        # Create the command list starting with Python and the script path
        command = ["python", script_path]
        # Add additional arguments from the params dictionary
        if params:
            for key, value in params.items():
                if value is not None:  # Check if the value is None
                    command.append(f"--{key}")
                    command.append(str(value))
                else:
                    command.append(f"--{key}")
        print("Command:", " ".join(command))

        # Start the subprocess
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Set the stdout to non-blocking
        fd = process.stdout.fileno()
        fl = fcntl.fcntl(fd, fcntl.F_GETFL)
        fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

        # Function to continuously output lines from a stream
        def stream_output(stream):
            while True:
                try:
                    line = stream.readline()
                    if line:
                        print(line, end='')
                    elif process.poll() is not None:
                        break
                except IOError as e:
                    # Ignore the error if no data is available yet
                    if e.errno != errno.EAGAIN and e.errno != errno.EWOULDBLOCK:
                        raise

        # Use a thread to capture the output stream
        output_thread = threading.Thread(target=stream_output, args=(process.stdout,))
        output_thread.start()

        # Wait for the subprocess to complete and the output thread to end
        process.wait()
        output_thread.join()

    else:
        print(f"Script not found: {script_path}")

In [ ]:
# Let's see what we can do with train.py
execute("train.py", {"help": None})

# Train and test CNN on CIFAR-10



## Parameters

In [ ]:
# parameters
batchsize = 32
seed      = 42
lr        = 3e-4
epochs    = 30
dataset   = "CIFAR-10"

## Train

In [ ]:
params = {
    "dataset": dataset,           # dataset name
    "batchsize": batchsize,       # batch size
    "seed": seed,                 # random seed
    "lr": lr,                     # learning rate
    "epochs": epochs              # number of epochs
}

# Execute 'train.py' with parameters
execute("train.py", params=params)

## Test 

In [ ]:
# parameters
params = {
    "model": f"models/model_{dataset}_{epochs-1:03d}.npz" # model name    
}

# Execute 'train.py' with parameters
execute("test.py", params=params)

# Train and test CNN on CIFAR-100

## Parameters

In [ ]:
# parameters
batchsize = 32
seed      = 42
lr        = 3e-4
epochs    = 30
dataset   = "CIFAR-100"

## Train

In [ ]:
params = {
    "dataset": dataset,           # dataset name
    "batchsize": batchsize,       # batch size
    "seed": seed,                 # random seed
    "lr": lr,                     # learning rate
    "epochs": epochs              # number of epochs
}

# Execute 'train.py' with parameters
execute("train.py", params=params)

## Test

In [ ]:
# parameters
params = {
    "dataset": dataset,           # dataset name
    "model": f"models/model_{dataset}_{epochs-1:03d}.npz" # model name    
}

# Execute 'train.py' with parameters
execute("test.py", params=params)